In [7]:
!pip install pulp sympy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import random
import numpy as np
import pulp
from sympy import *
import time

def get_nash(J):
        lines=range(len(J))
        columns=range(len(J[0]))

        lp = pulp.LpProblem('Nash', pulp.LpMaximize)
        V = pulp.LpVariable('V', cat='Continuous')

        a = pulp.LpVariable.dicts("a", (lines),  cat="Continous")
        lp.setObjective(V)

        lp += pulp.lpSum([a[i] for i in lines]) == 1

        for i in lines :
            lp +=a[i]>=0

        for j in columns :
            lp += pulp.lpSum([a[i]*J[i][j] for i in lines])-V>=0


        lp.solve()

        return [pulp.value(lp.objective), [pulp.value(a[i]) for i in lines]]

def initialiseV(V,m,n,tt):
    tm=(tt-1)/2
    for i in range(m):
        for j in range(n):
            V[i][j][0]=-1
            V[i][j][tt-1]=1

    for i in range(1,m):
        for k in range(1,tt-1): #le joueur 2 n'a pas de pierres
            if k+i>tm:
                V[i][0][k]=1
            if k+i==tm:
                V[i][0][k]=0
            if k+i<tm:
                V[i][0][k]=-1

    for j in range(1,n):          #le joueur 1 n'a pas plus de pierres
        for k in range(1,tt-1):
            if k-j<tm:
                V[0][j][k]=-1
            if k-j==tm:
                V[0][j][k]=0
            if k-j>tm:
                V[0][j][k]=1

    for k in range(1,tt-1):
        if k<tm:
            V[0][0][k]=-1
        if k==tm:
            V[0][0][k]=0
        if k>tm:
            V[0][0][k]=1




    return V

def initialiseV_b(V,m,n,tt): #dans cette fonction nous avons répondu à la question 4.b
    tm=(tt-1)/2
    for i in range(m):
        for j in range(n):
            V[i][j][0]=-1
            V[i][j][tt-1]=1

    for i in range(1,m):
        for k in range(1,tt-1): #le joueur 2 n'a pas de pierres
            if k+i>tm:
                V[i][0][k]=1
            if k+i==tm:
                V[i][0][k]=1
            if k+i<tm:
                V[i][0][k]=-1

    for j in range(1,n):          #le joueur 1 n'a pas plus de pierres
        for k in range(1,tt-1):
            if k-j<tm:
                V[0][j][k]=-1
            if k-j==tm:
                V[0][j][k]=1
            if k-j>tm:
                V[0][j][k]=1

    for k in range(1,tt-1):
        if k<tm:
            V[0][0][k]=-1
        if k==tm:
            V[0][0][k]=1
        if k>tm:
            V[0][0][k]=1




    return V

def initialiseV_c(V,m,n,tt):
    tm=(tt-1)/2
    for i in range(m):
        for j in range(n):
            V[i][j][0]=-1
            V[i][j][tt-1]=1

    for i in range(1,m):
        for k in range(1,tt-1): #le joueur 2 n'a pas de pierres
            if k+i>tm:
                V[i][0][k]=1
            if k+i==tm:
                V[i][0][k]=0
            if k+i<tm:
                V[i][0][k]=-1

    for j in range(1,n):          #le joueur 1 n'a pas plus de pierres
        for k in range(1,tt-1):
            if k-j<tm:
                V[0][j][k]=-1
            if k-j==tm:
                V[0][j][k]=0
            if k-j>tm:
                V[0][j][k]=1

    for k in range(1,tt-1):
        if k<tm:
            V[0][0][k]=-1
        if k==tm:
            V[0][0][k]=0
        if k>tm:
            V[0][0][k]=1




    return V

def construction1V(V,mm,nn,kk):

    J=np.zeros((mm)*(nn))
    J.resize((mm, nn))
    for i in range(mm):
        for j in range(nn):
            if i>j :
                J[i][j]=V[mm-i-1][nn-j-1][kk+1]
            if i<j :
                J[i][j]=V[mm-i-1][nn-j-1][kk-1]
            if i==j :
                J[i][j]=V[mm-i-1][nn-j-1][kk]


    qq=get_nash(J)
    V[mm][nn][kk]=qq[0]

    return [V,qq[1]]


def constructionV(m,n,tt):
    m=m+1
    n=n+1

    V=np.zeros(m*n*tt)
    V.resize((m, n, tt))
    V=initialiseV(V,m,n,tt)

    Tm=[]
    for i in range(1,m):
        Tn=[]
        for j in range(1,n):
            Tk=[]
            for k in range(1,tt-1):
                [V,L]=construction1V(V,i,j,k)
                Tk.append(L)

            Tn.append(Tk)

        Tm.append(Tn)

    return [V,Tm]


def Affiche(V,S,tt,m0,n0,t0,verbose=False):
    t=(int)((tt-1)/2+t0-1)
    t1=(int)((tt-1)/2+t0)
    if verbose:
      print ('le gain du joueur I dans Troll(',m0,',',n0,',',t0,') est : ',V[m0][n0][t1])
      print('la stratégie du joueur I dans Troll(',m0,',',n0,',',t0,') est : ', S[m0-1][n0-1][t])
    return S[m0-1][n0-1][t]




def isFinished(n, m, p, c): #Une fonction pour détérminer si la partie est finie 
    return n==0 or m==0 or p==c//2 or p==-1*(c//2)
        

def gameWinner(n, m, p, c): #Une fonction pour détérminer l'état du jeu
    if isFinished(n, m, p, c):
        if p == -1*(c//2):
            return -1
        elif p == c//2:
            return 1
        elif n== 0 :
            for i in range(m):
                p-= 1
            if p== 0:
                return 0
            return 1 if p> 0 else -1
        elif m== 0 :
            for i in range(n):
                p+= 1
            if p== 0:
                return 0
            return 1 if p> 0 else -1
    else:
        return -99

In [9]:
m=22    #nombre de pierres du joueur 1
n=22   #nombre de pierre du joueur 2
tt=9 #nombre de cases total incluant les chateaux (donc tt-2 cases effectives): la case centrale se trouve à la position (tt-1)/2

[V,S]=constructionV(m,n,tt)

In [10]:
m0=15   #<=m
n0=16  #<=n
t0=0 #position du troll une case à gauche (vers le joueur I) de la position centrale

Affiche(V,S,tt,m0,n0,t0, True)

le gain du joueur I dans Troll( 15 , 16 , 0 ) est :  -0.76593924
la stratégie du joueur I dans Troll( 15 , 16 , 0 ) est :  [0.24707163, 0.2754403, 0.47748807, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


[0.24707163,
 0.2754403,
 0.47748807,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [11]:
from tqdm import tqdm

def simulation():
    player1_record= []
    player2_record= []
    for i in tqdm(range(10000)):
        m0=20   #<=m
        n0=20  #<=n
        t0=0 #position du troll une case à gauche (vers le joueur I) de la position centrale

        while not isFinished(m0, n0, t0, tt):
            p= [prob*100 for prob in Affiche(V,S,tt,m0,n0,t0)]
            pp= random.choice(p)
            player1_move= p.index(pp)+1
            player2_move= random.choice([x for x in range(1, n0+1)])
            m0-= player1_move
            n0-= player2_move
            if player1_move==player2_move:
                t0= t0
            elif player1_move>player2_move:
                t0+= 1
            else:
                t0-= 1
            
        player1_record.append(gameWinner(m0, n0, t0, tt))
        player2_record.append(gameWinner(m0, n0, t0, tt)* -1)  
    print(" ")
    print("parties gagnates: ", len([x for x in player1_record if x==1]))
    print("parties perdues: ", len([x for x in player1_record if x==-1]))
    print("parties nulles", len([x for x in player1_record if x==0]))
simulation()

100%|██████████| 10000/10000 [00:00<00:00, 34307.80it/s]


 
parties gagnates:  9435
parties perdues:  530
parties nulles 35
